<a href="https://colab.research.google.com/github/fgith/data-science/blob/main/logistic-regression/logistic_regression_customerpersonality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget "https://raw.githubusercontent.com/fgith/data-science/main/datasets/marketing_campaign.csv"

--2022-04-16 04:54:32--  https://raw.githubusercontent.com/fgith/data-science/main/datasets/marketing_campaign.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217947 (213K) [text/plain]
Saving to: ‘marketing_campaign.csv’

marketing_campaign. 100%[===================>] 212.84K  --.-KB/s    in 0.03s   

2022-04-16 04:54:32 (8.18 MB/s) - ‘marketing_campaign.csv’ saved [217947/217947]



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np  

**Load CSV Dataset**

In [ ]:
dataset = pd.read_csv("marketing_campaign.csv",delimiter='\t')

In [142]:
df = pd.DataFrame(dataset)
df.shape

(2240, 29)

In [ ]:
df.isnull().sum()
# df.info()
# df.columns
# df.index
# df.values
# df.shape
# df.head()

ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Z_CostContact           0
Z_Revenue               0
Response                0
dtype: int64

**Assign X and y Data**

In [143]:
X = df.drop(columns = ['Year_Birth','ID','Dt_Customer'],axis=1)
y = X['Response']
X.shape,y.shape

((2240, 26), (2240,))

In [144]:
X['Marital_Status'].replace({"Single":1, "Together":2, "Married":3, "Divorced":4, "Widow":5, "Alone":6, "Absurd":7, "YOLO":8},inplace=True)
X['Education'].replace({"Graduation":1, "PhD":2, "Master":3,"Basic":4,"2n Cycle":5},inplace=True)
# X['Income'].fillna(0)

In [145]:
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(0, inplace=True)

In [146]:
from sklearn.preprocessing import StandardScaler

StandardScaler = StandardScaler()
X_scaled = pd.DataFrame(StandardScaler.fit_transform(X))
# X_scaled.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,-0.781341,-1.394226,0.251939,-0.825218,-0.929894,0.307039,0.983781,1.551577,1.679702,2.462147,...,0.693904,-0.28014,-0.28383,-0.28014,-0.262111,-0.11651,-0.097282,0.0,0.0,2.388846
1,-0.781341,-1.394226,-0.208699,1.032559,0.906934,-0.383664,-0.870479,-0.636301,-0.713225,-0.650449,...,-0.130463,-0.28014,-0.28383,-0.28014,-0.262111,-0.11651,-0.097282,0.0,0.0,-0.418612
2,-0.781341,-0.462802,0.778231,-0.825218,-0.929894,-0.798086,0.362723,0.570804,-0.177032,1.345274,...,-0.542647,-0.28014,-0.28383,-0.28014,-0.262111,-0.11651,-0.097282,0.0,0.0,-0.418612
3,-0.781341,-0.462802,-0.978043,1.032559,-0.929894,-0.798086,-0.870479,-0.560857,-0.651187,-0.503974,...,0.281720,-0.28014,-0.28383,-0.28014,-0.262111,-0.11651,-0.097282,0.0,0.0,-0.418612
4,0.014206,0.468623,0.257992,1.032559,-0.929894,1.550305,-0.389085,0.419916,-0.216914,0.155164,...,-0.130463,-0.28014,-0.28383,-0.28014,-0.262111,-0.11651,-0.097282,0.0,0.0,-0.418612


In [151]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled,y,train_size=0.7)

In [152]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train,y_train)

LogisticRegression()

In [153]:
y_pred = clf.predict(X_test)

In [156]:
from sklearn.metrics import mean_squared_error,r2_score,accuracy_score
accuracy = accuracy_score(y_test,y_pred)
print("Accuracy: %.2f" % accuracy_score(y_test,y_pred))
print("Mean Squared Error: %.2f" % mean_squared_error(y_test,y_pred))
print("Coefficient of Determination: %.2f" % r2_score(y_test,y_pred))

Accuracy: 1.00
Mean Squared Error: 0.00
Coefficient of Determination: 1.00
1.0
